In [16]:
# This notebook illustrates example outputs from Funman, and how to work with the ParameterSpace object it creates.

# The file scratch/hackathon/hackathon_fall_2023_demo_terarrium.py was used to generate the outputs rendered here.

SAVED_RESULTS_DIR = "./out"

# Import funman related code
import os
from funman.api.run import Runner
from funman_demo import summarize_results
from funman_demo.example.pde import plot_spatial_timeseries
from IPython.display import HTML
import matplotlib.pyplot as plt
import itertools
from funman import FunmanWorkRequest
import json
from funman.representation.constraint import LinearConstraint, ParameterConstraint, StateVariableConstraint
from funman.representation import Interval
from funman import FUNMANConfig
import logging
from funman_demo import summarize_results



RESOURCES = "../resources"
EXAMPLE_DIR = os.path.join(RESOURCES, "amr", "petrinet", "mira")
MODEL_PATH = os.path.join(
    EXAMPLE_DIR, "models", "BIOMD0000000955_askenet.json"
)
REQUEST_PATH = os.path.join(
    EXAMPLE_DIR, "requests", "BIOMD0000000955_askenet_request.json"
)


# %load_ext autoreload
# %autoreload 2

In [17]:
peak_time = Interval(lb=90, ub=110, closed_upper_bound=True)

peak_infections = Interval(lb=1e-2, ub=1e-1)
# ramp_up_infections = Interval(ub=0.1*peak_infections.ub)
# ramp_down_infections = Interval(ub=0.01*peak_infections.ub)
ramp_up_infections = Interval(lb=0.0, ub=0.5*peak_infections.ub)
ramp_down_infections = Interval(lb=0.0, ub=0.5*peak_infections.ub)


In [55]:
# Step 0: Unconstrained model, find any parameter assignment

with open(REQUEST_PATH, "r") as request:
    funman_request = FunmanWorkRequest.model_validate(json.load(request))
    results_unconstrained_point = Runner().run(
            MODEL_PATH,
            funman_request,
            description="SIDARTHE demo step 0",
            case_out_dir=SAVED_RESULTS_DIR,
    )
    # import cProfile

    # with cProfile.Profile() as pr:
summary = summarize_results(["Infected"], results_unconstrained_point, ylabel="Infected")
print(summary)
    #     pr.dump_stats("summary_profile.stats")

2024-02-21 01:15:34,270 - funman.server.worker - INFO - FunmanWorker running...
2024-02-21 01:15:34,273 - funman.server.worker - INFO - Starting work on: 8fdccfb6-957f-4685-9c5f-951d9ecabd78
2024-02-21 01:15:36,285 - funman.api.run - INFO - Dumping results to ./out/8fdccfb6-957f-4685-9c5f-951d9ecabd78.json
2024-02-21 01:15:39,034 - funman.scenario.consistency - INFO - 7{140}:	[+]
2024-02-21 01:15:39,038 - funman.server.worker - INFO - Completed work on: 8fdccfb6-957f-4685-9c5f-951d9ecabd78
2024-02-21 01:15:46,331 - funman.server.worker - INFO - Worker.stop() acquiring state lock ....
2024-02-21 01:15:46,626 - funman.server.worker - INFO - FunmanWorker exiting...
2024-02-21 01:15:46,628 - funman.server.worker - INFO - Worker.stop() completed.


In [56]:
# Take point from previous solution and define a box around it (used to label the box)
from funman import Box
from funman import ModelParameter, LABEL_ALL
point = results_unconstrained_point.parameter_space.true_points()[0]
# parameters = results_parameter_constraint.point_parameters(point)
parameters = results_unconstrained_point.model._parameter_names()
b = Box.from_point(point, radius = .000001, radius_vars=[p for p in parameters])
# b.bounds["epsilon"]
# b.bounds

with open(REQUEST_PATH, "r") as request:
    funman_request: FunmanWorkRequest = FunmanWorkRequest.model_validate(json.load(request))

    funman_request.parameters = [ModelParameter(name=p, interval=b.bounds[p], label=LABEL_ALL) for p in parameters]
    # print(funman_request.parameters[0]  )
    # funman_request.config.use_compartmental_constraints=False 
    results_synthesis_around_point = Runner().run(
            MODEL_PATH,
            funman_request,
            description="SIDARTHE demo",
            case_out_dir=SAVED_RESULTS_DIR,
            # dump_results=False
    )
    summary = summarize_results(["Infected"], results_synthesis_around_point, ylabel="Infected")
    print(summary)

2024-02-21 01:15:49,824 - funman.server.worker - INFO - FunmanWorker running...
2024-02-21 01:15:49,827 - funman.server.worker - INFO - Starting work on: b506b93f-7c9d-415b-87b4-4a8df15fdaf5
2024-02-21 01:15:51,834 - funman.api.run - INFO - Dumping results to ./out/b506b93f-7c9d-415b-87b4-4a8df15fdaf5.json
2024-02-21 01:15:52,912 - funman.search.box_search - INFO - progress: 0.12500
2024-02-21 01:15:53,295 - funman.search.box_search - INFO - progress: 0.25000
2024-02-21 01:15:53,649 - funman.search.box_search - INFO - progress: 0.37500
2024-02-21 01:15:53,987 - funman.search.box_search - INFO - progress: 0.50000
2024-02-21 01:15:54,307 - funman.search.box_search - INFO - progress: 0.62500
2024-02-21 01:15:54,621 - funman.search.box_search - INFO - progress: 0.75000
2024-02-21 01:15:54,920 - funman.search.box_search - INFO - progress: 1.00000
2024-02-21 01:15:56,017 - funman.server.worker - INFO - Completed work on: b506b93f-7c9d-415b-87b4-4a8df15fdaf5
2024-02-21 01:16:01,914 - funman.s

In [ ]:
# View profiling if present (must set `"profiling": True` in the config above)

import os
from IPython.core.display import Image
from IPython.display import display
profile_file = os.path.join(f"./summary_profile.stats")
if os.path.exists(profile_file):
    os.system(f"gprof2dot -f pstats {profile_file} | dot -Tpng -o summary-profile.png")
    display(Image(filename='summary-profile.png'))
else:
    print(f"{profile_file} file not present")

In [ ]:
# Step 1: Constrain peak infected, find any parameter assignment

with open(REQUEST_PATH, "r") as request:
    funman_request = FunmanWorkRequest.model_validate(json.load(request))

    
    funman_request.constraints = [
        StateVariableConstraint(name="peak_infected1", variable="Infected", interval=ramp_up_infections, timepoints=Interval(ub=peak_time.lb)),
        StateVariableConstraint(name="peak_infected2", variable="Infected", interval=peak_infections, timepoints=peak_time),
        StateVariableConstraint(name="peak_infected3", variable="Infected", interval=ramp_down_infections, timepoints=Interval(lb=peak_time.ub))
        ]

    results_constrained = Runner().run(
            MODEL_PATH,
            funman_request,
            description="SIDARTHE demo step 1",
            case_out_dir=SAVED_RESULTS_DIR,
    )
    summary = summarize_results(["Infected"], results_constrained, ylabel="Infected")
    print(summary)

In [ ]:
# Step 2: Constrain theta and epsilon so that 2 epsilon <= theta

with open(REQUEST_PATH, "r") as request:
    funman_request = FunmanWorkRequest.model_validate(json.load(request))

    epsilon = funman_request.parameter("epsilon")
    theta = funman_request.parameter("theta")
 

    funman_request.constraints = [
        StateVariableConstraint(name="peak_infected1", variable="Infected", interval=ramp_up_infections, timepoints=Interval(ub=peak_time.lb)),
        StateVariableConstraint(name="peak_infected2", variable="Infected", interval=peak_infections, timepoints=peak_time),
        StateVariableConstraint(name="peak_infected3", variable="Infected", interval=ramp_down_infections, timepoints=Interval(lb=peak_time.ub)),
        LinearConstraint(name="2_eps_le_theta", variables=[epsilon.name, theta.name], additive_bounds=Interval(lb=0), weights=[-2, 1])
        ]

    results_parameter_constraint = Runner().run(
            MODEL_PATH,
            funman_request,
            description="SIDARTHE demo step 2",
            case_out_dir=SAVED_RESULTS_DIR,
    )
    summary = summarize_results(["Infected"], results_parameter_constraint, ylabel="Infected")
    print(summary)

In [ ]:
# Step 3: Synthesize parameters


with open(REQUEST_PATH, "r") as request:
    funman_request = FunmanWorkRequest.model_validate(json.load(request))

    epsilon = funman_request.parameter("epsilon")
    theta = funman_request.parameter("theta")
    epsilon.label = "all"
    theta.label = "all"

    # for p in [epsilon]:
    #     p.interval = b.bounds[p.name]


    # adjustment_factor = 0.49
    # epsilon_adjustment = float(epsilon.width())*adjustment_factor
    # epsilon.interval = Interval(lb=epsilon.interval.lb+ epsilon_adjustment, ub=epsilon.interval.ub-epsilon_adjustment)

    # theta_adjustment = float(theta.width())*adjustment_factor
    # theta.interval = Interval(lb=theta.interval.lb+ theta_adjustment, ub=theta.interval.ub-theta_adjustment)

    # funman_request.config = FUNMANConfig(verbosity=5)

    funman_request.constraints = [
        StateVariableConstraint(name="peak_infected1", variable="Infected", interval=ramp_up_infections, timepoints=Interval(ub=peak_time.lb)),
        StateVariableConstraint(name="peak_infected2", variable="Infected", interval=peak_infections, timepoints=peak_time),
        StateVariableConstraint(name="peak_infected3", variable="Infected", interval=ramp_down_infections, timepoints=Interval(lb=peak_time.ub)),
        LinearConstraint(name="2_eps_le_theta", variables=[epsilon.name, theta.name], additive_bounds=Interval(lb=0), weights=[-2, 1])
        ]

    results_synthesis = Runner().run(
            MODEL_PATH,
            funman_request,
            description="SIDARTHE demo",
            case_out_dir=SAVED_RESULTS_DIR,
            # dump_results=False
    )
    summary = summarize_results(["Infected"], results_synthesis, ylabel="Infected")
    print(summary)

In [ ]:
# Take point from previous solution and define a box around it (used to label the box)
from funman import Box
point = results_parameter_constraint.parameter_space.true_points()[0]
# parameters = results_parameter_constraint.point_parameters(point)
parameters = results_parameter_constraint.model._parameter_names()
b = Box.from_point(point, radius = 0.01, radius_vars=[p for p in parameters])
# b.bounds["epsilon"]
b.bounds

In [ ]:
# Step 4: Synthesize all Parameters


from funman.constants import LABEL_ALL
from funman.representation.parameter import ModelParameter


with open(REQUEST_PATH, "r") as request:
    funman_request: FunmanWorkRequest = FunmanWorkRequest.model_validate(json.load(request))

    # epsilon = funman_request.parameter("epsilon")
    # theta = funman_request.parameter("theta")
    # epsilon.label = "all"
    # theta.label = "all"


    funman_request.parameters = [ModelParameter(name=p, interval=b.bounds[p], label=LABEL_ALL) for p in parameters]
        



    # adjustment_factor = 0.49
    # epsilon_adjustment = float(epsilon.width())*adjustment_factor
    # epsilon.interval = Interval(lb=epsilon.interval.lb+ epsilon_adjustment, ub=epsilon.interval.ub-epsilon_adjustment)

    # theta_adjustment = float(theta.width())*adjustment_factor
    # theta.interval = Interval(lb=theta.interval.lb+ theta_adjustment, ub=theta.interval.ub-theta_adjustment)

    # funman_request.config = FUNMANConfig(verbosity=5)

    funman_request.constraints = [
        StateVariableConstraint(name="peak_infected1", variable="Infected", interval=ramp_up_infections, timepoints=Interval(ub=peak_time.lb)),
        StateVariableConstraint(name="peak_infected2", variable="Infected", interval=peak_infections, timepoints=peak_time),
        StateVariableConstraint(name="peak_infected3", variable="Infected", interval=ramp_down_infections, timepoints=Interval(lb=peak_time.ub)),
        LinearConstraint(name="2_eps_le_theta", variables=[epsilon.name, theta.name], additive_bounds=Interval(lb=0), weights=[-2, 1])
        ]

    results_synthesis = Runner().run(
            MODEL_PATH,
            funman_request,
            description="SIDARTHE demo",
            case_out_dir=SAVED_RESULTS_DIR,
            # dump_results=False
    )
    summary = summarize_results(["Infected"], results_synthesis, ylabel="Infected")
    print(summary)